In [ ]:
import pandas as pd
import numpy as np
import seaborn

# Анализ данных при помощи Pandas
Помимо манипулций с таблицами, Pandas предоставляет широкие возможности для статистического анализа рядов и таблиц. 

Освежим в памяти содержимое датасета planets.

In [ ]:
planets = seaborn.load_dataset('planets')
planets.head()

## Series.describe()
У объектов `Series` существует замечательный метод `Series.describe()`. Он выдаёт основную информацию о распределении величины, представленной этим рядом:

In [ ]:
planets = seaborn.load_dataset('planets')

print('Взглянем на года открытия экзопланет: ')
planets['year'].describe()

Метод `describe` вернул информацию о 
* Количестве записей 
* Среднем возрасте
* Среднеквадратичном отклонение возраста
* Минимальный возраст
* Первый квартиль
* Медиана
* Третий квартиль
* Максимальный возраст

По этим данным можно сделать вывод, например, что за последние 4 года было открыто столько же планет, сколько за предыдущие 20. А так же данные о начале миссии.

## Series.unique()
Для первичного анализа датасета полезно узнать все значения категориальных признаков. Для этого можно воспользоваться методом `Series.unique()`, который вернёт множество значений:

In [ ]:
planets = seaborn.load_dataset('planets')
planets['method'].unique()

## Series.value_counts()

Ещё один полезный для анализа данных метод объекта `Series`, который позволяет подсчитать количество значений признака:

In [ ]:
planets = seaborn.load_dataset('planets')
planets['method'].value_counts()

Теперь мы знаем не только какие методы используются, но и какие из них самые популярные или продуктивные

# Группировка
Важная часть анализа данных - это их грамотное обобщение. При помощи аггрегирующих функций можно понять природу огромных массивов данных. 
Для обощения используется функция `DataFrame.groupby()` в качестве которой передаются колонка или `list` колонок, по которым делается группировка

In [ ]:
planets = seaborn.load_dataset('planets')
grouped = planets.groupby('year')

print(f'grouped: {grouped}')
print(f'type(grouped): {type(grouped)}')

Обратите внимание, что возвращаемое — не набор объектов `DataFrame` , а объект `DataFrameGroupBy`. 
Этот объект можно рассматривать как специальное представление объекта `DataFrame`, готовое к группировке, но не выполняющее никаких фактических вычислений до этапа применения агрегирования.

Чтобы получить результат, функция `DataFrame.groupby()` применяется с аггрегирующими функциями:
* count - подсчёт элементов
* min / max - минимальный/максимальный элемент
* mean / median / mode - среднее/медиана/мода
* std / var - среднеквадратичное отклонение / дисперсия

In [ ]:
planets = seaborn.load_dataset('planets')
planets.groupby('year').count().head(20)

Теперь попробуем по паре индексов:

In [ ]:
planets = seaborn.load_dataset('planets')

planets.groupby(['year', 'method']).count().head(20)

Порядок колонок важен:

In [ ]:
planets = seaborn.load_dataset('planets')

planets.groupby(['method', 'year']).count().head(20)

In [ ]:
Смена порядка в аргументе `DataFrame.groupby()` приводит к разным результатам, изменяя порядок группировки.

Однако, обратите внимание на индекс получившегося `DataFrame`. Он оказывается двухуровневым. И вообще оказывается объектом типа `MultiIndex`: 

In [ ]:
planets = seaborn.load_dataset('planets')

grouped = planets.groupby(['method', 'year']).count()
print(f'grouped.index = {grouped.index}')
print(f'\ntype(grouped.index) = {type(grouped.index)}',)

В обращении `MultiIndex` примерно такой же, как и знакомый нам ранее `Index`:

In [ ]:
planets = seaborn.load_dataset('planets')

grouped = planets.groupby(['method', 'year']).count()
print("grouped.loc['Microlensing'] = ")
grouped.loc['Microlensing']

In [ ]:
planets = seaborn.load_dataset('planets')

grouped = planets.groupby(['method', 'year']).count()
print("grouped.loc['Microlensing', 2004] = ")
grouped.loc['Microlensing', 2004]

Так же пытливый ум может заметить, что вроде как для аггрегации мы пользовались функцией `count()`, которая считает количество значений. 
Количество значений, по идее, должно совпадать с количеством записей, однако посмотрим ещё раз:

In [ ]:
planets = seaborn.load_dataset('planets')

grouped = planets.groupby(['method', 'year']).count()
grouped.loc['Microlensing']

Проверим, что же там такое:

In [ ]:
planets = seaborn.load_dataset('planets')
planets[planets['method'] == 'Microlensing'].head()

Делаем вывод, что count() не учитывает `NaN`

Так же помимо колонок, по которым происходит группировка, можно указывать колонки, по которым интересна аггрегация. 

Например, попытаемся извлечь информацию о том, на каких расстояниях работают те или иные методы обнаружения экзопланет. Выведем средние расстояния до звёздных систем для каждого из методов обнаружения:

In [ ]:
planets = seaborn.load_dataset('planets')
planets.groupby('method')['distance'].mean()

Для получения более развёрнутой информации можем воспользоваться набором аггрегационных функций. Для этого применяется фукнция `.agg()` в которой перечисляются аггрегирующие функции:

In [ ]:
planets = seaborn.load_dataset('planets')
planets.groupby('method')['distance'].agg(['min', 'mean', 'max'])

## Series.sort_values()
Так же для анализа бывает полезно упорядочить значения. Например, мы хотим знать о физических границах применимости методов:

In [ ]:
planets = seaborn.load_dataset('planets')
grouped = planets.groupby('method')['distance'].agg(['min', 'mean', 'max'])

grouped.sort_values('max')

Теперь мы знаем, что предел для астрометрии всего 20 парсек, а транзитный метод может обнаружить на расстоянии 8500 парсек. А это, на минуточку, 27 тысяч световых лет.